In [1]:
import nltk

# If haven't done so, go and download
# nltk.download()

In [2]:
import os

trend_files = [f.replace('.json', '') for f in os.listdir('../dataset/trends')]

In [3]:
import pandas as pd

tweet_df = pd.DataFrame([])

for i in trend_files:
    i = pd.read_json('../dataset/trends/'+i+'.json', orient='index')
    tweet_df = tweet_df.append(i)

In [4]:
tweet_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,metadata,source,...,favorite_count,favorited,retweeted,possibly_sensitive,lang,retweeted_status,inferred_text,quoted_status_id,quoted_status_id_str,quoted_status
0,2021-12-03 05:47:09+00:00,1466645158498238467,1466645158498238464,aku si study enthusiast ini abis daftar #TKOSu...,False,"[0, 202]","{'hashtags': [{'text': 'TKOSummit2021', 'indic...","{'media': [{'id': 1466645139644768258, 'id_str...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,0.0,in,None,aku si study enthusiast ini abis daftar #TKOSu...,NaN,NaN,NaN
1,2021-12-03 05:39:31+00:00,1466643239335723010,1466643239335723008,RT @CryptoizOffice: Kami selaku salah satu kom...,False,"[0, 140]","{'hashtags': [{'text': 'TKOSummit2021', 'indic...",None,"{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,NaN,in,{'created_at': 'Fri Dec 03 05:18:12 +0000 2021...,Kami selaku salah satu komunitas crypto di Ind...,NaN,NaN,NaN
2,2021-12-03 05:11:23+00:00,1466636158922678272,1466636158922678272,RT @cryptoizmedia: Kami selaku salah satu komu...,False,"[0, 140]","{'hashtags': [{'text': 'TKOSummit2021', 'indic...",None,"{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,NaN,in,{'created_at': 'Fri Dec 03 03:30:07 +0000 2021...,Kami selaku salah satu komunitas crypto di Ind...,NaN,NaN,NaN
3,2021-12-03 05:10:26+00:00,1466635920145219593,1466635920145219584,@Tokocrypto best boy nih sampe bikinin #TKOSum...,False,"[0, 164]","{'hashtags': [{'text': 'TKOSummit2021', 'indic...","{'media': [{'id': 1466635881335250949, 'id_str...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,0.0,in,None,@Tokocrypto best boy nih sampe bikinin #TKOSum...,NaN,NaN,NaN
4,2021-12-03 05:04:15+00:00,1466634361214623744,1466634361214623744,Pengen deh bikin thread soal crypto dan NFT ta...,False,"[0, 237]","{'hashtags': [{'text': 'TKOSummit2021', 'indic...","{'media': [{'id': 1466634343267270658, 'id_str...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,0,False,False,0.0,in,None,Pengen deh bikin thread soal crypto dan NFT ta...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,2021-12-02 21:09:14+00:00,1466514819683532804,1466514819683532800,Nampak bermasalah Fred dlm game ni. Mcm aku ca...,False,"[0, 238]","{'hashtags': [{'text': 'MUNARS', 'indices': [2...",None,"{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,0,False,False,NaN,in,None,Nampak bermasalah Fred dlm game ni. Mcm aku ca...,NaN,NaN,None
310,2021-12-02 21:08:59+00:00,1466514758547361799,1466514758547361792,"liat Club julukan Red Devil, Setan Merah, Emyu...",False,"[0, 131]","{'hashtags': [{'text': 'MUFC', 'indices': [118...",None,"{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,NaN,in,None,"liat Club julukan Red Devil, Setan Merah, Emyu...",NaN,NaN,None
311,2021-12-02 21:08:36+00:00,1466514661143052290,1466514661143052288,Jgm kalah seger #MUFC #MUNARS https://t.co/WWY...,False,"[0, 29]","{'hashtags': [{'text': 'MUFC', 'indices': [16,...",None,"{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,1,False,False,0.0,in,None,Jgm kalah seger #MUFC #MUNARS https://t.co/WWY...,1.466512e+18,1.466512e+18,{'created_at': 'Thu Dec 02 20:59:31 +0000 2021...
312,2021-12-02 21:08:09+00:00,1466514546449784839,1466514546449784832,"Emyu : ""Prof rangnick nyetadion, agak pressing...",False,"[0, 63]","{'hashtags': [{'text': 'mufc', 'indices': [58,...",None,"{'iso_language_code': 'in', 'result_ty

Goal: get sentiment. Therefore get only texts

In [5]:
tweets = pd.DataFrame([])
tweets['raw_text'] = tweet_df['inferred_text']
tweets.head()

,raw_text
0,aku si study enthusiast ini abis daftar #TKOSu...
1,Kami selaku salah satu komunitas crypto di Ind...
2,Kami selaku salah satu komunitas crypto di Ind...
3,@Tokocrypto best boy nih sampe bikinin #TKOSum...
4,Pengen deh bikin thread soal crypto dan NFT ta...


In [6]:
tweets.shape

(16028, 1)

In [7]:
# 1st: clear Twitter-specific terms
import re

def remove_twitter(text, for_gensim = False):
    twitter_stripped_text = re.sub('(@[A-Za-z0-9_]+)', '', text) if for_gensim else re.sub('(@)', '', text)   # Username removal
    twitter_stripped_text = re.sub(r'([http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*))', '', twitter_stripped_text)   # Link removal
    twitter_stripped_text = re.sub('(\#[A-Za-z]+)', '', twitter_stripped_text) if for_gensim else re.sub('(\#)', ' ', twitter_stripped_text)  # Hashtag removal, but preserving the word
    twitter_stripped_text = re.sub('([-&])', ' ', twitter_stripped_text)   # dash spacing, accommodating for better representation in spams
    return twitter_stripped_text

tweets['1st_clean_twitter'] = tweets['raw_text'].apply(lambda x: remove_twitter(x))
tweets['1st_clean_twitter_gensim'] = tweets['raw_text'].apply(lambda x: remove_twitter(x, for_gensim=True))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim
0,aku si study enthusiast ini abis daftar #TKOSu...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...
1,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...
2,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...
3,@Tokocrypto best boy nih sampe bikinin #TKOSum...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...
4,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...


In [8]:
# 2nd: remove punctuations
import string
punctuation_gensim = re.sub('([.,?!])','', string.punctuation)

def remove_punctuation(text, for_gensim=False):
    clean_text = "".join([i for i in text if i not in punctuation_gensim]) if for_gensim else "".join([i for i in text if i not in string.punctuation])
    clean_text = re.sub('([’])', "'", clean_text)
    return clean_text

tweets['2nd_punctuation'] = tweets['1st_clean_twitter'].apply(lambda x: remove_punctuation(x))
tweets['2nd_punctuation_gensim'] = tweets['1st_clean_twitter_gensim'].apply(lambda x: remove_punctuation(x, for_gensim=True))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim
0,aku si study enthusiast ini abis daftar #TKOSu...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...
1,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...
2,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...
3,@Tokocrypto best boy nih sampe bikinin #TKOSum...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...
4,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...


In [9]:
import emoji

def remove_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text)

tweets['3rd_emoji'] = tweets['2nd_punctuation'].apply(lambda x: remove_emoji(x))
tweets['3rd_emoji_gensim'] = tweets['2nd_punctuation_gensim'].apply(lambda x: remove_emoji(x))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim
0,aku si study enthusiast ini abis daftar #TKOSu...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...
1,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...
2,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...
3,@Tokocrypto best boy nih sampe bikinin #TKOSum...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...
4,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...


In [10]:
tweets['4th_tokenized'] = tweets['3rd_emoji'].apply(nltk.tokenize.word_tokenize)
tweets['4th_tokenized_gensim'] = tweets['3rd_emoji_gensim'].apply(nltk.tokenize.word_tokenize)
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_tokenized,4th_tokenized_gensim
0,aku si study enthusiast ini abis daftar #TKOSu...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...,aku si study enthusiast ini abis daftar TKOSu...,aku si study enthusiast ini abis daftar 2021 d...,"[aku, si, study, enthusiast, ini, abis, daftar...","[aku, si, study, enthusiast, ini, abis, daftar..."
1,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,"[Kami, selaku, salah, satu, komunitas, crypto,...","[Kami, selaku, salah, satu, komunitas, crypto,..."
2,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,Kami selaku salah satu komunitas crypto di Ind...,"[Kami, selaku, salah, satu, komunitas, crypto,...","[Kami, selaku, salah, satu, komunitas, crypto,..."
3,@Tokocrypto best boy nih sampe bikinin #TKOSum...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...,Tokocrypto best boy nih sampe bikinin TKOSumm...,best boy nih sampe bikinin 2021 biar kita ngg...,"[Tokocrypto, best, boy, nih, sampe, bikinin, T...","[best, boy, nih, sampe, bikinin, 2021, biar, k..."
4,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,Pengen deh bikin thread soal crypto dan NFT ta...,"[Pengen, deh, bikin, thread, soal, crypto, dan...","[Pengen, deh, bikin, thread, soal, crypto, dan..."


In [13]:
# On the other project that I made I chose stopwords because I was going for bag-of-words approach.
crf = nltk.CRFTagger()
crf.set_model_file('../resources/pos_tag/all_indo_man_tag_corpus_model.crf.tagger')

_ = crf.tag_sents(tweets['4th_tokenized'])

In [14]:
_

[[('aku', 'NN'),
  ('si', 'NN'),
  ('study', 'FW'),
  ('enthusiast', 'FW'),
  ('ini', 'PR'),
  ('abis', 'VB'),
  ('daftar', 'IN'),
  ('TKOSummit2021', 'NNP'),
  ('dari', 'IN'),
  ('Tokocrypto', 'NNP'),
  ('dong', 'VB'),
  ('wkwkkwk', 'NN'),
  ('biar', 'CD'),
  ('nggak', 'NN'),
  ('ketinggalan', 'NN'),
  ('soal', 'NN'),
  ('crypto', 'NN'),
  ('dan', 'CC'),
  ('NFT', 'NNP'),
  ('yg', 'NNP'),
  ('lagi', 'RB'),
  ('naik', 'VB'),
  ('daun', 'FW'),
  ('banget', 'FW'),
  ('ituuuu', 'FW'),
  ('cus', 'FW'),
  ('daftar', 'FW'),
  ('gengs', 'FW')],
 [('Kami', 'PRP'),
  ('selaku', 'IN'),
  ('salah', 'JJ'),
  ('satu', 'CD'),
  ('komunitas', 'NN'),
  ('crypto', 'NN'),
  ('di', 'IN'),
  ('Indonesia', 'NNP'),
  ('sangat', 'RB'),
  ('mendukung', 'VB'),
  ('movement', 'FW'),
  ('ini', 'PR'),
  ('dan', 'CC'),
  ('mendukung', 'VB'),
  ('TKOSummit2021', 'CD'),
  ('yang', 'SC'),
  ('akan', 'MD'),
  ('diselelenggarakan', 'VB'),
  ('besok', 'NN'),
  ('Set', 'FW'),
  ('your', 'FW'),
  ('alarm', 'FW'),
  ('and'

From here we can see that the POS tagger we use did not tag a bit of the text correctly. Another solution that we have is to load an established Indonesian corpus (from Flair), train the model ourselves, and use it as our POS tagger.

In [ ]:
import flair
corpus_ind = flair.data_fetcher.NLPTask